In [3]:
import requests

## Web Check

In [ ]:
requests.get("http://127.0.0.1:2443")

In [36]:
requests.get("http://127.0.0.1:2443", verify=False)

KeyboardInterrupt: 

In [37]:
requests.get("http://127.0.0.1:28080").text

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=28080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f257e87c850>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [68]:
requests.get("http://127.0.0.1:18080").text

'Hello world'

In [69]:
requests.get("http://127.0.0.1:18080/count").text

'Request count: 5'

## Pipe Check

In [ ]:
import os
import time
import signal
CTL_READ_FD = 198
STATE_WRITE_FD = 199

In [ ]:

ctl_rfd, ctl_wfd = os.pipe()
state_rfd, state_wfd = os.pipe()
pid = os.fork()
if pid == 0:
    os.dup2(ctl_rfd, CTL_READ_FD)
    os.dup2(state_wfd, STATE_WRITE_FD)
    os.close(ctl_rfd)
    os.close(ctl_wfd)
    os.close(state_rfd)
    os.close(state_wfd)
    KERN="/home/czx/openbmc-workspace/openbmc/build/romulus/tmp/deploy/images/romulus"
    os.execve('/usr/bin/gdbserver', 
                ['/usr/bin/gdbserver', '127.0.0.1:1234',
                '/home/czx/qemu-workspace/qemu-spy/build/qemu-system-arm',
                '-m', '256M', '-M', 'romulus-bmc',
                '-drive', f'file={KERN}/obmc-phosphor-image-romulus.static.mtd,if=mtd,format=raw',
                '-net', 'nic',
                '-net', 'user,hostfwd=:127.0.0.1:2222-:22,hostfwd=:127.0.0.1:2443-:443,hostfwd=:127.0.0.1:18080-:8080,hostfwd=udp:127.0.0.1:2623-:623,hostname=qemu',
                '-d', 'plugin',
                '-plugin', '../build/libaflspy.so',
                '-D', 'qemu_log.txt',
                '-nographic'], {})

In [ ]:
os.read(state_rfd, 5)

In [ ]:
os.write(ctl_wfd, b"fork")

In [ ]:
s = os.read(state_rfd, 5)
if s == b'ready':
    print("子进程: ", s)
    os.write(ctl_wfd, b'good\n')

In [ ]:
os.write(ctl_wfd, b'stop')